In [ ]:
import os
import re
import sys
import shutil

from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [6, 6]


In [ ]:
def mkdir_if_necessary(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def exists_with_postfix(path, postfix=".jpg"):
    return os.path.exists(path + postfix)


In [ ]:
def sort_overviews(d):
    files = next(os.walk(d))[2]

    ov_d = os.path.join(d, 'overviews')
    mkdir_if_necessary(ov_d)

    p = re.compile('.*?field.*?sted.*?')

    for f in files:
        if not re.match(p, f):
            shutil.move(os.path.join(d, f), os.path.join(ov_d, f))        

In [ ]:
### 1: set the directory to process
#dir_to_process = os.path.join(os.getcwd(), 'AutomatedAcquisitions')
dir_to_process = '/Users/david/Desktop/5th_shipment_20161028/mixed_150s_B/'

In [ ]:
### 2: move all the overview files into a separate folder
dirs = [d for d in next(os.walk(dir_to_process))[1] if not d.startswith('.')]
print(dirs)

for d in dirs:
    sort_overviews(os.path.join(dir_to_process, d))

In [ ]:
### 3: set subfolder to process
d = os.path.join(dir_to_process, 'K562_150sec_B')

In [ ]:
### 4: SORTING into good/bad/mediocre


gd_d = os.path.join(d, 'good')
bd_d = os.path.join(d, 'bad')
md_d = os.path.join(d, 'mediocre')

mkdir_if_necessary(gd_d)
mkdir_if_necessary(bd_d)
mkdir_if_necessary(md_d)

files = [f for f in next(os.walk(d))[2] if f.endswith('.msr')]

for fi in files:
    f = os.path.join(d,fi)
    im = Image.open(f + ".jpg")
    plt.imshow(im)
    plt.show()
    print('-----')
    sys.stdout.flush()
    decision = input("ISGOOD? [(g)ood/(b)ad/(m)ediocre] :")
    dec = decision.upper()[0]
    
    # print(dec == "G")
    
    if dec == "G":
        shutil.move(os.path.join(d, fi), os.path.join(gd_d, fi))
        shutil.move(os.path.join(d, fi + ".jpg"), os.path.join(gd_d, fi + ".jpg"))
    elif dec == "B":
        shutil.move(os.path.join(d, fi), os.path.join(bd_d, fi))
        shutil.move(os.path.join(d, fi + ".jpg"), os.path.join(bd_d, fi + ".jpg"))
    elif dec == "M":
        shutil.move(os.path.join(d, fi), os.path.join(md_d, fi))
        shutil.move(os.path.join(d, fi + ".jpg"), os.path.join(md_d, fi + ".jpg"))
    else:
        pass
